# Steps
1. Read an image in RGB
1. Apply the M transformation
1. Compute chromaticity coordinates for r and b
1. Project (log b, log r) into a one dimensional value along the angled line
1. See the grayscale image along the angled line

In [1]:
import numpy as np
from moviepy.editor import VideoFileClip
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
%matplotlib qt
%matplotlib inline

In [6]:
imagefile = '../k9-train/180909/HanDep/Straight/IMG_0003_MOV/frame1022.jpg'
myimg = cv2.imread(imagefile)
myimg = cv2.cvtColor(myimg, cv2.COLOR_BGR2RGB)

In [3]:
myimg.shape

(720, 1280, 3)

In [4]:
M = [0.9988,-0.0669,-0.0001,-0.0497,0.9886,-0.0001,-0.0043,-0.1346,1.0]
M = np.array(M).reshape(3,3)

array([[  9.98800000e-01,  -6.69000000e-02,  -1.00000000e-04],
       [ -4.97000000e-02,   9.88600000e-01,  -1.00000000e-04],
       [ -4.30000000e-03,  -1.34600000e-01,   1.00000000e+00]])

In [7]:
myimg[0,0,:]

array([217, 253, 253], dtype=uint8)

In [8]:
out = M*myimg[0,0,:]

In [9]:
out

array([[  2.16739600e+02,  -1.69257000e+01,  -2.53000000e-02],
       [ -1.07849000e+01,   2.50115800e+02,  -2.53000000e-02],
       [ -9.33100000e-01,  -3.40538000e+01,   2.53000000e+02]])